# 0. Libraries and API-Key

In [43]:
import os
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from openai import OpenAI
from IPython.display import Markdown

import evaluate

In [14]:
API_KEY = input("Enter your API-Key")

# 1. Dataset Selection and Loading

In [ ]:
# Load the document
document_dir = "data/"
filename = "BiologicalReviews-2024-Kershenbaum-Automaticdetectionforbioacousticresearchapracticalguidefromandfor.pdf"
file_path = os.path.join(document_dir, filename)

# Use PyPDFLoader to load the PDF
pages = PyPDFLoader(file_path).load_and_split()
print(f"Loaded {len(pages)} pages from the document.")

# Initialize CharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=10)

# Split documents
docs = text_splitter.split_documents(pages)

Loaded 57 pages from the document.


# 2. Exploratory Data Analysis (EDA)

# 3. RAG

### 3.1 Embedding

In [17]:
# Create the embeddings function
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=API_KEY) # You can find the different models in openai.com, under Embeddings

### 3.2 Vector Store

In [24]:
# Load it into Chroma
db = Chroma.from_documents(documents=docs,
                           embedding=embeddings,
                           persist_directory="./chroma_db"
                           )

### 3.3 Retrieval

In [ ]:
# User question
user_question = "How can I transform acoustic data?"

# Build the context
def build_context(question):
    retrieved_docs = db.similarity_search(query=question, # Consider similarity_search_with_score to chose k depending on the distance between each consecutive doc 
                                      k=5 # Number of relevant retrieved docs
                                      )
    context = "\n\n".join([doc.page_content for doc in retrieved_docs]) # Combine documents into a single string to prepare the context
    return context

# Build the prompt
def build_prompt(question, context):
    prompt = f"""
    ## INTRODUCTION
    You are a Chatbot designed to help answer technical questions based on information in a scientific document.

    ## USER QUESTION
    The user asked: "{question}"

    ## CONTEXT
    Technical Documentation for the software:
    '''
    {context}
    '''

    ## RESTRICTIONS
    - Use only factual information from the provided context.
    - If you cannot find the answer, state clearly that you do not know based on the available context.
    - Focus strictly on bioacoustics-related content and avoid any information outside the provided document.
    - Avoid speculation, opinions, and subjectivity.
    - Keep the response formal, clear, and concise, without unnecessary details.

    ## TASK
    1. Directly answer the user's question.
    2. Mention specific sections or page numbers if relevant information is available in particular parts of the document.
    3. Format the answer in Markdown.

    ## RESPONSE STRUCTURE:
    '''
    # [Answer Title]
    [answer text]



    ## CONVERSATION:
    User: {question}
    Agent:
    """
    return prompt

In [68]:
context = build_context(user_question)
prompt = build_prompt(user_question, context)

### 3.4 Generation

In [ ]:
client = OpenAI(api_key = API_KEY)

# Prepare the messages payload
messages = [{'role': 'user', 'content': prompt}]

# Set model parameters
model_params = {'model': "gpt-4o-mini",
                'temperature': 0.4,
                'max_tokens': 3000
                }

chat_completion = client.chat.completions.create(messages = messages,
                                                 **model_params,
                                                 timeout=120
                                                 )

answer = chat_completion.choices[0].message.content
display(Markdown(answer))

# Transforming Acoustic Data
To transform acoustic data, one can utilize data augmentation techniques to artificially increase variability in the dataset. The choice of augmentation techniques should be aligned with the specific application and must cover the range of variations found in real signals. However, it is crucial to avoid transformations that may invalidate annotations, such as reversing sounds in a bird call detector, which could lead to confusion between species calls. 

Additionally, the dominant approach in bioacoustics involves using spectral representations like spectrograms or mel-spectrograms, which facilitate the visualization of acoustic data and enable the use of vision-based models such as Convolutional Neural Networks (CNNs). It is important to note that while these representations are useful, some information from the raw waveform may be lost, particularly for transient signals.

Source:
• Based on content from pages 1-2 in the document titled "Automatic Detection for Bioacoustic Research: A Practical Guide"

# 4. Evaluation

In [ ]:
# Load test questions
test_questions = pd.read_csv("data/bioacoustic_questions_answers.csv")
test_questions = test_questions[:5]
test_questions

,Question,Answer
0,What is the importance of automatic detection ...,Automatic detection in bioacoustic research pl...
1,How does automatic detection assist in monitor...,Automatic detection facilitates ecosystem moni...
2,What challenges are faced in implementing auto...,Challenges in implementing automatic detection...
3,How do machine learning algorithms contribute ...,Machine learning algorithms analyze vast amoun...
4,What are some real-world applications of autom...,Automatic detection is widely applied in conse...


In [69]:
# Generate answers to the questions
predictions = []
for question in test_questions["Question"]:
    context = build_context(question)
    prompt = build_prompt(question,context)
    messages = [{'role': 'user', 'content': prompt}]
    chat_completion = client.chat.completions.create(messages = messages,
                                                    **model_params,
                                                    timeout=120
                                                    )
    predictions.append(chat_completion.choices[0].message.content)
predictions = pd.Series(predictions)

In [ ]:
# Check visually by human
with pd.option_context('display.max_colwidth', None):
    display(predictions, test_questions["Answer"])

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         # Importance of Automatic Detection in Bioacoustic Research\n\

0                                        Automatic detection in bioacoustic research plays a vital role by enabling researchers to analyze vast amounts of audio data efficiently. It allows for the identification of species, tracking population density, and observing behavioral patterns without direct human observation. This technology has revolutionized ecological studies by making passive monitoring possible, especially in remote or otherwise inaccessible environments, thus contributing to conservation efforts and biodiversity studies.
1                                     Automatic detection facilitates ecosystem monitoring by analyzing acoustic indices that reflect biodiversity and environmental changes. Through continuous audio data collection, this technology can identify shifts in species occupancy, variations in vocal activity, and even detect the presence of invasive species. This approach aids in assessing ecosystem health and can alert researchers to changes that may indicate

In [73]:
# Compare the predictions with the real answers
rouge = evaluate.load('rouge')

test_results = rouge.compute(predictions=predictions,
                                       references=test_questions["Answer"],
                                       use_aggregator=True,
                                       use_stemmer=True
                                       )

In [74]:
test_results

{'rouge1': 0.3009109577638095,
 'rouge2': 0.06685732525035737,
 'rougeL': 0.16918138706750935,
 'rougeLsum': 0.19027186916605593}

# 5. 